<a href="https://colab.research.google.com/github/YuliaVodopyanova/professional_profiles_parcer/blob/main/profiles_of_psychologists_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парсинг

In [ ]:
!pip install fake-useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import ast
# from fake_useragent import UserAgent

In [ ]:
ua = UserAgent()

base_url = 'https://www.b17.ru'

#психологи онлайн:
urls = [f'https://www.b17.ru/online/?page={page}' for page in range(1, 31)]

#психологи в Санкт-Петербурге
urls_sbp = [f'https://www.b17.ru/psiholog/sankt-peterburg/?page={page}' for page in range(1, 31)]

#психологи в Москве:
urls_mos = [f'https://www.b17.ru/psiholog/moskva/?page={page}' for page in range(1, 31)]

#психологи в Екатеринбурге
urls_ekb = [f'https://www.b17.ru/psiholog/ekaterinburg/?page={page}' for page in range(1, 31)]

#психологи в Краснодаре
urls_krd = [f'https://www.b17.ru/psiholog/krasnodar/?page={page}' for page in range(1, 31)]

#психологи в Новосибирске
urls_nvs = [f'https://www.b17.ru/psiholog/novosibirsk/?page={page}' for page in range(1, 31)]

Сбор ссылок на анкеты психологов:

In [ ]:
professional_pages = []

counter = 0

for url in urls_ekb: #urls, urls_sbp,...
  headers = {
        'User-Agent': ua.random
        }

  response = requests.get(url, headers=headers)

  if response.status_code == 200:

    html_soup = BeautifulSoup(response.content, 'html.parser')

    page_links = html_soup.find_all('a', class_='h', href=True)

    for link in page_links:
      professional_pages.append(base_url + link['href'])

  time.sleep(random.randrange(2, 7))

In [ ]:
len(professional_pages)

290

Сохранение списков с ссылками во избежание потери информации при обновлении среды выполнения кода и для их последующей обработки:

In [ ]:
with open('links_for_final_mos.txt', 'w') as file:
  print(professional_pages, file=file)



---



Открытие собранных ссылок: открываем текствые файлы (с ссылками на информацию о психологах, которые работают в разных городах и которые принимают пациентов онлайн), проверяем, нет ли повторяющихся анкет (дубликатов)

In [ ]:
import ast

file = open('links_for_final.txt', 'r')
online_professional_pages = ast.literal_eval(file.read())

file = open('/content/links_for_final_spb.txt', 'r')
professional_pages_spb = ast.literal_eval(file.read())

file = open('/content/links_for_final_mos.txt', 'r')
professional_pages_mos = ast.literal_eval(file.read())

file = open('/content/links_for_final_ekb.txt', 'r')
professional_pages_ekb = ast.literal_eval(file.read())

file = open('/content/links_for_final_krd.txt', 'r')
professional_pages_krd = ast.literal_eval(file.read())

file = open('/content/links_for_final_nvs.txt', 'r')
professional_pages_nvs = ast.literal_eval(file.read())

In [ ]:
all_links = online_professional_pages + professional_pages_spb + \
professional_pages_mos + professional_pages_ekb + professional_pages_krd + \
professional_pages_nvs

all_professional_links = sorted(set(all_links), key=all_links.index)

print(f'Количество дубликатов: {len(all_links) - len(all_professional_links)}\n'
      f'Всего ссылок на уникальные анкеты: {len(all_professional_links)}')

Количество дубликатов: 137
Всего ссылок на уникальные анкеты: 1632


Наличие дубликатов свидельствует о том, что многие психологи предлагают свои услуги как офлайн, так и онлайн.

Сохраняем ссылки на анкеты психологов без дубликатов:

In [ ]:
with open('all_professional_links.txt', 'w') as file:
  print(all_professional_links, file=file)

Сбор текстов каждой из анкет:

In [ ]:
authors_data = []

for link in professional_pages:
  headers = {
        'User-Agent': ua.random
        }
  response = requests.get(link, headers=headers)

  if response.status_code == 200:

    html_article = BeautifulSoup(response.content, 'html.parser')

    heading = html_article.h1

    text_div = html_article.find('div', style="overflow: hidden; word-wrap: break-word; line-height: 130%; margin-top: -2px; z-index: 2; position: relative;")

    if text_div and heading:
      author = heading.text
      author_text = text_div.get_text()
      authors_data.append({"author": author, "text": author_text})

    time.sleep(random.randrange(2, 7))



---



Сохраняем информацию:

In [ ]:
with open('till_299id_inclusive.txt', 'w') as file:
  print(authors_data, file=file)

# Создание датафрейма и предобработка текстов

In [ ]:
import ast

In [ ]:
file = open('/content/till_1631id_inclusive.txt', 'r')
data = ast.literal_eval(file.read())

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df

,author,text,link
0,Кузьмичев Александр Сергеевич,Получить консультацию1424 обращения\n4 в этом ...,https://www.b17.ru/kuzmichev/
1,Илья Ахмедов,Получить консультацию770 обращений\n12 в этом ...,https://www.b17.ru/jason/
2,Татьяна Колесникова,Получить консультацию1124 обращения\n5 в этом ...,https://www.b17.ru/tkolesnikova/
3,Вьюков Антон Сергеевич,Получить консультацию2907 обращений\n23 в этом...,https://www.b17.ru/a_vyukov/
4,Гуэдова Светлана Леонидовна,Получить консультацию1635 обращений\n40 в этом...,https://www.b17.ru/svetlana_guedova/
...,...,...,...
1627,Жеребцова Кристина Александровна,Получить консультациюЗдравствуйте! Я клиническ...,https://www.b17.ru/kristina_zh/
1628,Наталья Савчук,Получить консультацию2 обращения\nМеня зовут Н...,https://www.b17.ru/savchuk6699/
1629,Чиркова Полина Андреевна,Получить консультациюОсновное направление моей...,https://www.b17.ru/polinachirkovaa/
1630,Минина Анна Юрьевна,"Получить консультациюМеня зовут Анна Минина, в...",https://www.b17.ru/bubenkova_telesnik/


In [ ]:
def clean_start(text):
  """Удаляет начало текста, которое начинается с "Получить консультацию"
     и "в этом месяце ". Предобрабатывает основные моменты
  """
  text = re.sub(r'\.([а-яА-Я0-9])', r'. \1', text)
  text = re.sub(r'([.,;?!:"])([А-Я])', r'\1 \2', text)
  text = re.sub(r'([а-я])([А-Я])', r'\1 \2', text)
  text = text.replace('-', ' ')


  pattern = r'^Получить консультацию\d+ обращени[й|я|е]\n\d+ в '\
    r'этом месяце\n|^Получить консультацию\d+ обращени[й|я]\n|'\
    r'^Получить консультацию'

  match = re.match(pattern, text)

  if match:
    start_index = match.end()
    text = text[start_index:]

  text = re.sub(r'(\d+)', r' \1 ', text)
  text = text.replace('( ', '(').replace(' )', ')').replace('• ', ' •')\
  .replace('т.д.', 'тд.').replace('т.п.', 'тп.').replace(')', ') ')\
  .replace('   ', ' ').replace('  ', ' ')

  return text

In [ ]:
df['text_clean_start'] = df['text'].apply(clean_start)

df

,author,text,link,text_clean_start,lemmas
0,Кузьмичев Александр Сергеевич,Получить консультацию1424 обращения\n4 в этом ...,https://www.b17.ru/kuzmichev/,Я врач психотерапевт. Я занимаюсь психокоррекц...,я врач психотерапевт я заниматься психокоррекц...
1,Илья Ахмедов,Получить консультацию770 обращений\n12 в этом ...,https://www.b17.ru/jason/,"Приветствую Вас! Меня зовут Илья Ахмедов, прак...",приветствовать вы я звать илья ахмедов практик...
2,Татьяна Колесникова,Получить консультацию1124 обращения\n5 в этом ...,https://www.b17.ru/tkolesnikova/,"Квалификация: психолог, когнитивно поведенческ...",квалификация психолог когнитивный поведенчески...
3,Вьюков Антон Сергеевич,Получить консультацию2907 обращений\n23 в этом...,https://www.b17.ru/a_vyukov/,Квалификация: Дипломированный медицинский (кли...,квалификация дипломированный медицинский клини...
4,Гуэдова Светлана Леонидовна,Получить консультацию1635 обращений\n40 в этом...,https://www.b17.ru/svetlana_guedova/,Рада приветствовать вас на своей странице! Бла...,рад приветствовать вы на свой страница благода...
...,...,...,...,...,...
1627,Жеребцова Кристина Александровна,Получить консультациюЗдравствуйте! Я клиническ...,https://www.b17.ru/kristina_zh/,"Здравствуйте! Я клинический психолог, использ...",здравствовать я клинический психолог использо...
1628,Наталья Савчук,Получить консультацию2 обращения\nМеня зовут Н...,https://www.b17.ru/savchuk6699/,"Меня зовут Наталья, я дипломированный психолог...",я звать наталья я дипломированный психолог кон...
1629,Чиркова Полина Андреевна,Получить консультациюОсновное направление моей...,https://www.b17.ru/polinachirkovaa/,Основное направление моей работы классическая...,основной направление мой работа классически...
1630,Минина Анна Юрьевна,"Получить консультациюМеня зовут Анна Минина, в...",https://www.b17.ru/bubenkova_telesnik/,"Меня зовут Анна Минина, в прошлом Бубенкова. ...",я звать анна минин в прошлый бубенков я серти...


In [ ]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~«»'

In [ ]:
!pip install pymystem3

In [ ]:
from pymystem3 import Mystem
from string import punctuation

punctuation += '«»()[]'

m = Mystem()


def lemmatize(text):
    text = ''.join([c for c in text if c not in punctuation])
    lemmas = m.lemmatize(text.lower())
    return ''.join(lemmas)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
df['lemmas'] = df['text_clean_start'].apply(lemmatize)
df

,author,text,link,text_clean_start,lemmas
0,Кузьмичев Александр Сергеевич,Получить консультацию1424 обращения\n4 в этом ...,https://www.b17.ru/kuzmichev/,Я врач психотерапевт. Я занимаюсь психокоррекц...,я врач психотерапевт я заниматься психокоррекц...
1,Илья Ахмедов,Получить консультацию770 обращений\n12 в этом ...,https://www.b17.ru/jason/,"Приветствую Вас! Меня зовут Илья Ахмедов, прак...",приветствовать вы я звать илья ахмедов практик...
2,Татьяна Колесникова,Получить консультацию1124 обращения\n5 в этом ...,https://www.b17.ru/tkolesnikova/,"Квалификация: психолог, когнитивно поведенческ...",квалификация психолог когнитивный поведенчески...
3,Вьюков Антон Сергеевич,Получить консультацию2907 обращений\n23 в этом...,https://www.b17.ru/a_vyukov/,Квалификация: Дипломированный медицинский (кли...,квалификация дипломированный медицинский клини...
4,Гуэдова Светлана Леонидовна,Получить консультацию1635 обращений\n40 в этом...,https://www.b17.ru/svetlana_guedova/,Рада приветствовать вас на своей странице! Бла...,рад приветствовать вы на свой страница благода...
...,...,...,...,...,...
1627,Жеребцова Кристина Александровна,Получить консультациюЗдравствуйте! Я клиническ...,https://www.b17.ru/kristina_zh/,"Здравствуйте! Я клинический психолог, использ...",здравствовать я клинический психолог использо...
1628,Наталья Савчук,Получить консультацию2 обращения\nМеня зовут Н...,https://www.b17.ru/savchuk6699/,"Меня зовут Наталья, я дипломированный психолог...",я звать наталья я дипломированный психолог кон...
1629,Чиркова Полина Андреевна,Получить консультациюОсновное направление моей...,https://www.b17.ru/polinachirkovaa/,Основное направление моей работы классическая...,основной направление мой работа классический ...
1630,Минина Анна Юрьевна,"Получить консультациюМеня зовут Анна Минина, в...",https://www.b17.ru/bubenkova_telesnik/,"Меня зовут Анна Минина, в прошлом Бубенкова. ...",я звать анна минин в прошлый бубенков я серти...


# Общие сведения о датафрейме. Подсчет и поиск терминов в датафрейме.

##Сведения

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/diploma/preprocessed_profiles_df.csv')

In [ ]:
df.head(5)

,author,text,link,text_clean_start,lemmas
0,Кузьмичев Александр Сергеевич,Получить консультацию1424 обращения\n4 в этом ...,https://www.b17.ru/kuzmichev/,Я врач психотерапевт. Я занимаюсь психокоррекц...,я врач психотерапевт я заниматься психокоррекц...
1,Илья Ахмедов,Получить консультацию770 обращений\n12 в этом ...,https://www.b17.ru/jason/,"Приветствую Вас! Меня зовут Илья Ахмедов, прак...",приветствовать вы я звать илья ахмедов практик...
2,Татьяна Колесникова,Получить консультацию1124 обращения\n5 в этом ...,https://www.b17.ru/tkolesnikova/,"Квалификация: психолог, когнитивно поведенческ...",квалификация психолог когнитивный поведенчески...
3,Вьюков Антон Сергеевич,Получить консультацию2907 обращений\n23 в этом...,https://www.b17.ru/a_vyukov/,Квалификация: Дипломированный медицинский (кли...,квалификация дипломированный медицинский клини...
4,Гуэдова Светлана Леонидовна,Получить консультацию1635 обращений\n40 в этом...,https://www.b17.ru/svetlana_guedova/,Рада приветствовать вас на своей странице! Бла...,рад приветствовать вы на свой страница благода...


In [ ]:
print(f'Количество собранных анкет: {len(df)}')
total_words = sum(len(text) for text in df['text_clean_start'])
print(f'Общее количество слов в анкетах: {total_words}')
print(f'Среднее количество слов в анкете: {total_words/len(df)}')

Количество собранных анкет: 1632
Общее количество слов в анкетах: 4649946
Среднее количество слов в анкете: 2849.231617647059


In [ ]:
sum(len(text) for text in df['text_clean_start'])

4649946

##Поиск вхождений терминов

In [ ]:
from pymystem3 import Mystem

m = Mystem()

In [ ]:
def count_word_occurrences(df, column_name, word):
    """
  Подсчитывает количество вхождений слова в текстах столбца DataFrame.

  Args:
    df: DataFrame, в котором нужно подсчитать вхождения слова.
    column_name: Название столбца DataFrame с текстами.
    word: Слово, которое нужно посчитать.

  Returns:
    Tuple: количество документов, содержащих слово, общее количество вхождений слова.
  """

    df_filtered = df[df[column_name].notna()]
    df_filtered[column_name] = df_filtered[column_name].astype(str)
    docs_with_word = df_filtered[df_filtered[column_name].str.contains(word)]

    doc_count = len(docs_with_word)
    word_count = sum([text.count(word) for text in df_filtered[column_name]])

    return doc_count, word_count


words = ['конформность', 'панический атака', 'психосоматика', 'психоз', 'обсессия', 'альтруизм', 'внутриличностный конфликт', 'паранойя', 'шизофрения', 'конгруэнтность',
         'фрустрация', 'рефлексия', 'аффект', 'социопатия', 'прокрастинация', 'эмпатия', 'ипохондрия', ' фобия',
         'инсайт', 'нарциссизм']

vals = []

for word in words:
    doc_count, word_count = count_word_occurrences(df, 'lemmas', word)

    vals.append(doc_count)

    print(f'Количество документов, содержащих "{word.upper()}": {doc_count}')
    print(f'Общее количество вхождений "{word.upper()}": {word_count}')
    print(f'IPM: {(word_count/total_words)*1000000}')
    print()

Количество документов, содержащих "КОНФОРМНОСТЬ": 0
Общее количество вхождений "КОНФОРМНОСТЬ": 0
IPM: 0.0

Количество документов, содержащих "ПАНИЧЕСКИЙ АТАКА": 426
Общее количество вхождений "ПАНИЧЕСКИЙ АТАКА": 525
IPM: 112.90453695591303

Количество документов, содержащих "ПСИХОСОМАТИКА": 206
Общее количество вхождений "ПСИХОСОМАТИКА": 265
IPM: 56.98990913012753

Количество документов, содержащих "ПСИХОЗ": 14
Общее количество вхождений "ПСИХОЗ": 17
IPM: 3.6559564347628983

Количество документов, содержащих "ОБСЕССИЯ": 5
Общее количество вхождений "ОБСЕССИЯ": 7
IPM: 1.5053938260788404

Количество документов, содержащих "АЛЬТРУИЗМ": 1
Общее количество вхождений "АЛЬТРУИЗМ": 1
IPM: 0.2150562608684058

Количество документов, содержащих "ВНУТРИЛИЧНОСТНЫЙ КОНФЛИКТ": 48
Общее количество вхождений "ВНУТРИЛИЧНОСТНЫЙ КОНФЛИКТ": 50
IPM: 10.75281304342029

Количество документов, содержащих "ПАРАНОЙЯ": 1
Общее количество вхождений "ПАРАНОЙЯ": 1
IPM: 0.2150562608684058

Количество документов, соде

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE,SIG_DFL)

<Handlers.SIG_IGN: 1>

In [ ]:
def get_sentences_around_lemma(df, lemma, sentences_before=2, sentences_after=2):
  """
  Выводит предложения из колонки с текстом, окружающие указанную лемму.

  """

  sentences = []
  # current_sentence = []
  # sentence_count = 0

  for i, row in df.iterrows():
    if len(lemma.split()) == 1:
      if lemma in row['lemmas']:

          text = nltk.sent_tokenize(row.text_clean_start)


          for j, sentence in enumerate(text):
              lemmas = m.lemmatize(sentence.lower())

              for lem in lemmas:

                if lemma == lem:
                  start = max(0, j - sentences_before)
                  end = min(len(text), j + sentences_after + 1)
                  res = [' '.join(text[start:end]), row.author, row.link, 1]

                  sentences.append(res)

    if len(lemma.split()) == 2:
      if lemma in row['lemmas']:
          text = nltk.sent_tokenize(row.text_clean_start)


          for j, sentence in enumerate(text):
              lemmas = m.lemmatize(sentence.lower())

              if all(item in lemmas for item in m.lemmatize(lemma)):

                start = max(0, j - sentences_before)
                end = min(len(text), j + sentences_after + 1)
                res = [' '.join(text[start:end]), row.author, row.link, 1]

                sentences.append(res)

  return sentences


sentences = get_sentences_around_lemma(df, 'фрустрация', sentences_before=1, sentences_after=1)


last_link = None
i = 0

list_with_word = []

for ind, sentence in enumerate(sentences):
    if sentence[2] != last_link:
        i += 1
        last_link = sentence[2]
        print(f'{i}. {sentence[0]} [{sentence[1]}, URL: {sentence[2]}]', end='\n\n')
    else:
        sentence[3] = sentences[ind-1][3] + 1
        print(f'{i}.{sentence[3]} {sentence[0]} [{sentence[1]}, URL: {sentence[2]}]', end='\n\n')
    list_with_word.append(sentence[0])

print('Overall uses:', len(sentences))
print('IPM:', (len(sentences)/total_words)*1000000)

# ['конформность', 'паническая атака', 'психосоматика', 'психоз', 'обсессия', 'альтруизм', 'внутриличностный конфликт', 'паранойя', 'шизофрения', 'конгруэнтность',
#          'фрустрация', 'рефлексия', 'аффект', 'социопатия', 'прокрастинация', 'эмпатия', 'ипохондрия', ' фобия',
#          'инсайт', 'нарциссизм']

# [9+0, 199+409, 59+203, 1400+14, 15+5, 426+1, 4+46, 878+1, 1661+38,
#                3+2, 185+4, 1356+7, 780+5, 25+0, 22+180, 248+53, 64+28, 982+323,
#                79+23, 248+9]

1. Психокоррекция подразумевает тренировку навыков проживания эмоций, эмоциональной самомотивации, фильтров восприятия. А также проработку спонтанных мыслей и состояния фрустрации, повышение толерантности в внешним жизненным препятствия и внутренним переживаниям. Рекомендую вам следующие статьи и видео по теме:https://www.b 17 .ru/article/negativnaya_programma_zhizni/ https://www.b 17 .ru/article/kak_zastavit_sebya_chto to_sdelat/https://www.b 17 .ru/blog/nevroz_ili_depressiya/https://www.b 17 .ru/article/kak_snyat_vnutrennij_kontrol/ Показать видео
Показать видео
Показать видео
Обсессивно компульсивное расстройство Подразумевает наличие навязчивых мыслей, воспоминаний и/или сочетанных с ними навязчивых движений и действий (ритуалов) , которые возникают на пике внутреннего напряжения. [Кузьмичев Александр Сергеевич, URL: https://www.b17.ru/kuzmichev/]

1.2 Работа с этой проблемой занимает в среднем 15 18 недель в зависимости от тяжести заболевания, не/использования и эффекта препаратов

#N-gram analysis

In [ ]:
import re
from nltk import ngrams
from collections import Counter

In [ ]:
!pip3 install spacy
!python3 -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 103.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize


nlp = spacy.load("ru_core_news_sm")
stop_words = set(stopwords.words('russian'))


def lemmatize_text(text):
    doc = nlp(text)
    result = " ".join([token.lemma_ for token in doc])
    return result

def get_ngrams(text, n):
    words = word_tokenize(text.lower())
    return list(ngrams(words, n))

def is_alpha_ngram(ngram):
    return all(word.isalpha() for word in ngram)

def contains_stop_word(ngram):
    return any(word in stop_words for word in ngram)

def analyze_texts(texts: str, word: str):
    sentences = re.split(r'\n+', texts.strip())

    all_ngrams = []

    for sentence in sentences:
        all_ngrams.extend(get_ngrams(sentence.lower(), 2)) #setting n for ngram

    filtered_ngram_freq = {
        ngram: freq for ngram, freq in Counter(all_ngrams).items()
        if word in lemmatize_text(' '.join(ngram)) and is_alpha_ngram(ngram) and not contains_stop_word(ngram)
    }

    sorted_ngram_frequencies = sorted(filtered_ngram_freq.items(), key=lambda item: item[1])

    print('\nЧастотность n-грамм:')
    for ngram_tuple in sorted_ngram_frequencies:
        print(f"{' '.join(ngram_tuple[0])}: {ngram_tuple[1]}")

analyze_texts('\n'.join(list_with_word), 'фрустрация')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



Частотность n-грамм:
состояния фрустрации: 1
преодоления фрустрации: 1


In [ ]:
df.to_csv('preprocessed_profiles_df.csv', index = False)